In [1]:
import json
import requests
import pandas as pd

In [2]:
metadata_api = "https://maayanlab.cloud/sigcom-lincs/metadata-api"

In [3]:
payload = {
    "filter": {
        "where": {
            "meta": {
                "fullTextSearch": "Chemical Perturbation 2021"
            }
        }
    }
}

res = requests.post(metadata_api + "/libraries/find", json=payload)
res.json()

chem_pert_dataset = res.json()[0]
dataset = chem_pert_dataset['dataset']

In [4]:
libid = chem_pert_dataset["id"]
payload = {
    "filter": {
        "where": {
            "meta.pert_name": "dexamethasone",
            "library": libid
        }
    }
}

res = requests.post(metadata_api + "/signatures/find", json=payload)
dex_signatures = res.json()

In [6]:
len(dex_signatures)

467

In [35]:
table = pd.DataFrame([dex_signatures[0]['meta']])
for i in range(1, len(dex_signatures)):
    row = pd.DataFrame([dex_signatures[i]['meta']])
    table = pd.concat([table, row])

In [37]:
table.reset_index()

,index,md5,doid,sha256,tissue,anatomy,cmap_id,disease,version,filename,...,pert_type,$validator,data_level,pubchem_id,creation_time,persistent_id,size_in_bytes,uncompressed_size_in_bytes,$counter,$download_counter
0,0,f8550c4ae170d84a47b2e315d75bfc64,DOID:1324,3c4397b55647dea09adaa12853f102ceafdf5ca120ad8c...,lung,UBERON:0002048,LKCP002_A549_48H:E08,lung cancer,1,L1000_LINCS_DCIC_LKCP002_A549_48H_E08_dexameth...,...,Chemical,https://raw.githubusercontent.com/MaayanLab/si...,5,5743,2021-05-19,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...,217006,217006,NaN,NaN
1,0,2bae292caf243ab06cdca3d23e060937,DOID:1324,b3affb611b35f05d83e2942f2724aac78c39fae18121e9...,lung,UBERON:0002048,LKCP002_A549_48H:E07,lung cancer,1,L1000_LINCS_DCIC_LKCP002_A549_48H_E07_dexameth...,...,Chemical,https://raw.githubusercontent.com/MaayanLab/si...,5,5743,2021-05-19,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...,217203,217203,NaN,NaN
2,0,e18ffc8a744b1db9a2faa77b0ff21073,DOID:1324,eb7b7b6d55ffa3b7449fc032cc0f76e936788c4b2a856d...,lung,UBERON:0002048,LKCP002_A549_24H:E08,lung cancer,1,L1000_LINCS_DCIC_LKCP002_A549_24H_E08_dexameth...,...,Chemical,https://raw.githubusercontent.com/MaayanLab/si...,5,5743,2021-05-19,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...,216777,216777,NaN,NaN
3,0,bd9869e44fcf0e72e9b4629cf1a59e95,DOID:1324,378fbed02e85b3421aea70b7b7224f955a7d2be5bcb4f5...,lung,UBERON:0002048,LKCP002_A549_6H:E09,lung cancer,1,L1000_LINCS_DCIC_LKCP002_A549_6H_E09_dexametha...,...,Chemical,https://raw.githubusercontent.com/MaayanLab/si...,5,5743,2021-05-19,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...,217204,217204,NaN,NaN
4,0,7c016384dc50ba68fb56184c071c2641,DOID:1324,2c25590f9a53fb807623976f7108a2f088759070891043...,lung,UBERON:0002048,LKCP002_A549_6H:E07,lung cancer,1,L1000_LINCS_DCIC_LKCP002_A549_6H_E07_dexametha...,...,Chemical,https://raw.githubusercontent.com/MaayanLab/si...,5,5743,2021-05-19,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...,217082,217082,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,0,8cbd9e21980dd5fcb4adb3c9dc6f453e,DOID:4001,bc95abbbb41476e463233b53d751f2287585c1010bbfbf...,ovary,UBERON:0000992,CPC006_EFO27_6H:BRD-A35108200-001-04-7:10,ovarian carcinoma,1,L1000_LINCS_DCIC_CPC006_EFO27_6H_F09_dexametha...,...,Chemical,https://raw.githubusercontent.com/MaayanLab/si...,5,5743,2021-05-08,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...,345103,345103,NaN,NaN
463,0,1e7732b55c746e622276ef9b57c90442,DOID:1324,ad65aafb2d7bb4d93efa47c60d708ab18ea4f05d6bdf1f...,lung,UBERON:0002048,CPC006_DV90_6H:BRD-A69951442-001-01-3:10,lung cancer,1,L1000_LINCS_DCIC_CPC006_DV90_6H_O03_dexamethas...,...,Chemical,https://raw.githubusercontent.com/MaayanLab/si...,5,5743,2021-05-08,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...,345162,345162,NaN,NaN
464,0,61480526868312cd2af17d580986193b,DOID:1324,829cdeb1e23d197a1b90d399b1dd6ae5cde880ec60bbd0...,lung,UBERON:0002048,CPC006_CORL23_6H:BRD-A35108200-001-04-7:10,lung cancer,1,L1000_LINCS_DCIC_CPC006_CORL23_6H_F09_dexameth...,...,Chemical,https://raw.githubusercontent.com/MaayanLab/si...,5,5743,2021-05-08,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...,345096,345096,NaN,NaN
465,0,7d56548a73857feb71f9d61ae0fdfcf3,DOID:1324,56e3e6cf111dcd007ad4f6bf08b79c0ebdebb25b48c971...,lung,UBERON:0002048,CPC006_H1299_6H:BRD-A69951442-001-01-3:10,lung cancer,1,L1000_LINCS_DCIC_CPC006_H1299_6H_O03_dexametha...,...,Chemical,https://raw.githubusercontent.com/MaayanLab/si...,5,5743,2021-05-08,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...,345550,345550,NaN,NaN


In [38]:
table.columns

Index(['md5', 'doid', 'sha256', 'tissue', 'anatomy', 'cmap_id', 'disease',
       'version', 'filename', 'local_id', 'cell_line', 'pert_dose',
       'pert_name', 'pert_time', 'pert_type', '$validator', 'data_level',
       'pubchem_id', 'creation_time', 'persistent_id', 'size_in_bytes',
       'uncompressed_size_in_bytes', '$counter', '$download_counter'],
      dtype='object')

In [40]:
table[['tissue', 'disease','local_id', 'cell_line','pert_dose','pert_name','pert_time','pert_type','pubchem_id','creation_time']]

,tissue,disease,local_id,cell_line,pert_dose,pert_name,pert_time,pert_type,pubchem_id,creation_time
0,lung,lung cancer,LKCP002_A549_48H_E08_dexamethasone_1.11uM,A549,1.11 uM,dexamethasone,48 h,Chemical,5743,2021-05-19
0,lung,lung cancer,LKCP002_A549_48H_E07_dexamethasone_10uM,A549,10 uM,dexamethasone,48 h,Chemical,5743,2021-05-19
0,lung,lung cancer,LKCP002_A549_24H_E08_dexamethasone_1.11uM,A549,1.11 uM,dexamethasone,24 h,Chemical,5743,2021-05-19
0,lung,lung cancer,LKCP002_A549_6H_E09_dexamethasone_0.12uM,A549,0.12 uM,dexamethasone,6 h,Chemical,5743,2021-05-19
0,lung,lung cancer,LKCP002_A549_6H_E07_dexamethasone_10uM,A549,10 uM,dexamethasone,6 h,Chemical,5743,2021-05-19
...,...,...,...,...,...,...,...,...,...,...
0,ovary,ovarian carcinoma,CPC006_EFO27_6H_F09_dexamethasone_10uM,EFO27,10 uM,dexamethasone,6 h,Chemical,5743,2021-05-08
0,lung,lung cancer,CPC006_DV90_6H_O03_dexamethasone_10uM,DV90,10 uM,dexamethasone,6 h,Chemical,5743,2021-05-08
0,lung,lung cancer,CPC006_CORL23_6H_F09_dexamethasone_10uM,CORL23,10 uM,dexamethasone,6 h,Chemical,5743,2021-05-08
0,lung,lung cancer,CPC006_H1299_6H_O03_dexamethasone_10uM,H1299,10 uM,dexamethasone,6 h,Chemical,5743,2021-05-08


In [21]:
'''API_URL = "https://maayanlab.cloud/sigcom-lincs/metadata-api/signatures/value_count"

payload = {
    "where": {
        "meta.pert_name":  "dexamethasone"
    }
}

res = requests.get(API_URL, params={"filter": json.dumps(payload)})
results = res.json()

print(json.dumps(results, indent=2))'''

'API_URL = "https://maayanlab.cloud/sigcom-lincs/metadata-api/signatures/value_count"\n\npayload = {\n    "where": {\n        "meta.pert_name":  "dexamethasone"\n    }\n}\n\nres = requests.get(API_URL, params={"filter": json.dumps(payload)})\nresults = res.json()\n\nprint(json.dumps(results, indent=2))'